# Install Libraries 
I have installed locally the packages 

In [1]:
!pip install -q /kaggle/input/keras-lib-dataset/keras_nlp-0.9.2-py3-none-any.whl --no-deps

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/keras-lib-dataset/keras_nlp-0.9.2-py3-none-any.whl'



# Import important libraries 

In [ ]:
# /kaggle/input/ai-mathematical-olympiad-prize/sample_submission.csv
# /kaggle/input/ai-mathematical-olympiad-prize/AIMO Prize - Note on Language and Notation.pdf
# /kaggle/input/ai-mathematical-olympiad-prize/train.csv
# /kaggle/input/ai-mathematical-olympiad-prize/test.csv
# /kaggle/input/ai-mathematical-olympiad-prize/aimo/competition.cpython-310-x86_64-linux-gnu.so
# /kaggle/input/ai-mathematical-olympiad-prize/aimo/__init__.py
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import Datasets 

In [ ]:
train = pd.read_csv("/kaggle/input/ai-mathematical-olympiad-prize/train.csv") 
test = pd.read_csv("/kaggle/input/ai-mathematical-olympiad-prize/test.csv")

# Pre-Trained Models Configuration

In [ ]:
# Pretrained Model
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Loading the  model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
) 

model.config.use_cache = False
model.config.pretraining_tp = 1

# Defining LoRA parameters (Adapter Layer )
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
)
# Attach trainable adapters
model = get_peft_model(model, peft_params)

# Training Parameters
training_params = TrainingArguments(
    output_dir="./CodeLlama2Movies",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_steps=10,
    optim="paged_adamw_32bit",
    logging_strategy="steps",
    save_steps=25,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="tensorboard"
)

# Model fine-tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_params,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_params,
    packing= False,
)

trainer.train()

# Save the model
trainer.save_model(new_model)
tokenizer.save_pretrained(new_model)